In [1]:
import sys
import math
from tqdm import tqdm
sys.path.insert(0, '../')

import torch
from torch import nn
from torch.nn import functional as F

from attention import MultiHeadAttention
from encoder import Encoder
from decoder import Decoder
from positional_encoding import PositionalEncoder
from transformer import Transformer

In [2]:
VOCAB_SIZE = 15
D_MODEL = 6
NUM_HEADS = 2
MAX_LEN = 10
BATCH_SIZE = 8

In [3]:
sample_input_batch = torch.randint(1, VOCAB_SIZE-1, (BATCH_SIZE, MAX_LEN))
sample_output_batch = torch.randint(1, VOCAB_SIZE-1, (BATCH_SIZE, MAX_LEN))
sample_output_batch[:, -1] = VOCAB_SIZE-1
sample_output_batch[:, 0] = 0

sample_input_batch.size(), sample_output_batch.size()

(torch.Size([8, 10]), torch.Size([8, 10]))

In [4]:
transformer = Transformer(vocab_size=VOCAB_SIZE, max_len=MAX_LEN, d_model=D_MODEL, num_heads=NUM_HEADS)

In [5]:
from torchsummary import summary

In [10]:
summary(transformer, sample_input_batch, sample_output_batch)

RuntimeError: CUDA out of memory. Tried to allocate 570.00 MiB (GPU 0; 4.00 GiB total capacity; 2.60 GiB already allocated; 275.40 MiB free; 2.62 GiB reserved in total by PyTorch)

In [117]:
input_embeder = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=D_MODEL)
output_embeder = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=D_MODEL)
encoder = Encoder(d_model=D_MODEL, num_heads=NUM_HEADS, max_len=MAX_LEN)
decoder = Decoder(d_model=D_MODEL, num_heads=NUM_HEADS)

In [118]:
sample_input_embedded = input_embeder(sample_input_batch)
sample_output_embedded = output_embeder(sample_output_batch)

In [119]:
sample_input_embedded.size(), sample_output_embedded.size()

(torch.Size([8, 10, 6]), torch.Size([8, 10, 6]))

In [120]:
encoder_output = encoder(sample_input_embedded)
encoder_output.size()

torch.Size([8, 10, 6])

In [121]:
sample_input_embedded.size()

torch.Size([8, 10, 6])

In [122]:
decoder_output = decoder(encoder_output, sample_output_embedded)
decoder_output.size()

torch.Size([8, 10, 6])

In [123]:
linear = nn.Linear(in_features=D_MODEL, out_features=VOCAB_SIZE)

In [124]:
sample_output_batch.size()

torch.Size([8, 10])

In [128]:
linear_output = linear(decoder_output)

In [134]:
linear_output[0].size() # 하나의 시퀀스, 각 단어를 15(vocab size)차원으로 맵핑 후 소프트맥스

torch.Size([10, 15])

In [142]:
output = F.softmax(linear_output, dim=-1)

In [146]:
torch.argmax(output, dim=-1)

tensor([[ 8,  8,  9,  8, 11, 11, 11,  8,  8,  8],
        [ 8,  8,  8, 14,  9,  8, 11,  8, 11,  7],
        [ 8,  8,  8,  7,  9,  3,  8,  8,  8,  8],
        [ 8,  8,  8,  8, 11, 11, 11, 11, 11,  7],
        [ 7, 11,  9,  7, 11, 11,  8,  8, 11,  8],
        [ 8, 10, 13,  9, 10,  8, 11, 11, 11,  7],
        [11,  4,  3,  9,  8,  8,  8,  9,  8,  8],
        [ 9, 10,  9, 10,  8,  8, 11,  8, 11,  8]])